## Day 20 作業：將 missing 值替代為其他值

由於現實世界中的資料可能有缺漏，因此資料中就會有 missing 值存在，而在資料科學及機器學習中，缺漏值的處理關係到分析或預測的結果。

在今天的作業中，請將資料集中的年齡缺漏值，依性別分別替換為資料集中男性或女性的年齡平均值。

本次作業使用的資料集：Kaggle Titanic: Machine Learning from Disaster 的 train 資料集

競賽主頁：[https://www.kaggle.com/c/titanic/overview](https://www.kaggle.com/c/titanic/overview)

請自行下載資料集 **train.csv**：[https://www.kaggle.com/c/titanic/data](https://www.kaggle.com/c/titanic/data)

In [1]:
using DataFrames, CSV

In [2]:
df = CSV.read("data//train.csv")
first(df, 5)

,PassengerId,Survived,Pclass,Name
,Int64,Int64,Int64,String
1,1,0,3,"Braund, Mr. Owen Harris"
2,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Thayer)"
3,3,1,3,"Heikkinen, Miss. Laina"
4,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)"
5,5,0,3,"Allen, Mr. William Henry"


In [3]:
size(df)

(891, 12)

呼叫 `describe()` 函式時，加上 `:nmissing` 參數，顯示各 column 的 missing 值數目。

可以看到 Age 共有 177 個缺漏值。

In [4]:
describe(df, :nmissing)

,variable,nmissing
,Symbol,Union…
1,PassengerId,
2,Survived,
3,Pclass,
4,Name,
5,Sex,
6,Age,177
7,SibSp,
8,Parch,
9,Ticket,


計算男性及女性的平均年齡。

【提示】使用 `skipmissing()` 去掉缺漏值後，再計算平均值。

In [5]:
using Statistics

In [6]:
unique(df[:, :Sex])

2-element Array{String,1}:
 "male"  
 "female"

In [7]:
df[df.Sex .== "male", :][:, :Age]

577-element Array{Union{Missing, Float64},1}:
 22.0     
 35.0     
   missing
 54.0     
  2.0     
 20.0     
 39.0     
  2.0     
   missing
 35.0     
 34.0     
 28.0     
   missing
  ⋮       
 26.0     
 33.0     
 47.0     
 20.0     
 19.0     
   missing
 33.0     
 28.0     
 25.0     
 27.0     
 26.0     
 32.0     

In [8]:
m = skipmissing(df[df.Sex .== "male", :][:, :Age])

Base.SkipMissing{Array{Union{Missing, Float64},1}}(Union{Missing, Float64}[22.0, 35.0, missing, 54.0, 2.0, 20.0, 39.0, 2.0, missing, 35.0  …  47.0, 20.0, 19.0, missing, 33.0, 28.0, 25.0, 27.0, 26.0, 32.0])

In [9]:
male_age_avg = mean(skipmissing(m))

30.72664459161148

In [10]:
f = skipmissing(df[df.Sex .== "female", :][:, :Age])

Base.SkipMissing{Array{Union{Missing, Float64},1}}(Union{Missing, Float64}[38.0, 26.0, 35.0, 27.0, 14.0, 4.0, 58.0, 14.0, 55.0, 31.0  …  27.0, 47.0, 28.0, 15.0, 56.0, 25.0, 22.0, 39.0, 19.0, missing])

In [11]:
female_age_avg = mean(skipmissing(f))

27.915708812260537

將 Age column 中的 missing 值，依性別替換為上面計算出來的平均年齡值。

【提示】可產生新的 column 來存放新的結果，而不取代掉原先的 Age column。

In [12]:
df.Age_new = df[:, :Age]

891-element Array{Union{Missing, Float64},1}:
 22.0     
 38.0     
 26.0     
 35.0     
 35.0     
   missing
 54.0     
  2.0     
 27.0     
 14.0     
  4.0     
 58.0     
 20.0     
  ⋮       
 56.0     
 25.0     
 33.0     
 22.0     
 28.0     
 25.0     
 39.0     
 27.0     
 19.0     
   missing
 26.0     
 32.0     

In [13]:
df.Age_new[df.Sex .== "male"]   = coalesce.(df[df.Sex .== "male", :Age_new], male_age_avg)
df.Age_new[df.Sex .== "female"] = coalesce.(df[df.Sex .== "female", :Age_new], female_age_avg)

314-element Array{Float64,1}:
 38.0              
 26.0              
 35.0              
 27.0              
 14.0              
  4.0              
 58.0              
 14.0              
 55.0              
 31.0              
 27.915708812260537
 15.0              
  8.0              
  ⋮                
 27.915708812260537
 42.0              
 27.0              
 47.0              
 28.0              
 15.0              
 56.0              
 25.0              
 22.0              
 39.0              
 19.0              
 27.915708812260537

In [14]:
df[1:10, [:Sex,:Age, :Age_new]]

,Sex,Age,Age_new
,String,Float64⍰,Float64⍰
1,male,22.0,22.0
2,female,38.0,38.0
3,female,26.0,26.0
4,female,35.0,35.0
5,male,35.0,35.0
6,male,missing,30.7266
7,male,54.0,54.0
8,male,2.0,2.0
9,female,27.0,27.0
